In [ ]:
knitr::opts_chunk$set(echo = F,include = TRUE,tidy = TRUE,cache = FALSE,eval = TRUE, message = FALSE,warning = FALSE,fig.align = "center")

In [ ]:
library(knitr)
library(pander)
library(xtable)
library(stargazer)
#setwd("C:\\Users\\qtckp\\OneDrive\\Desktop\\Powerlift\\Estimating RM")

#Загрузка данных####
library(tidyverse)
library(magrittr)
library(ggformula)
library(ggthemes)
library(tidyquant)
library(ggvis)
library(plotrix)
library(car)
#library(DAAG)
library(leaps)
library(plotly)

data=read_tsv("data(rus).tsv",
              skip=1,col_names = F,na="",
              col_types = "fddnfffnnnff",
              comment="#"
              ) %>% tbl_df()

colnames(data)=c("Date","RM","MRM","Count","Action","Sex","Experience","Age","Weight","Height","BodyType","Mail")
data %<>%#filter(Count<=20) %>% 
  arrange(MRM,Count,Weight) %>%  mutate(
  CountGroup=cut(Count,breaks = c(1,3,6,10,20,40)),
  AgeGroup=cut(Age,breaks = c(1,19,27,35,70)),
  Experience=factor(Experience,levels = c("До двух лет","2-3 года","4-5 лет","6-10 лет","11-15 лет" ,"больше 15 лет"),ordered = T),
  Index=Weight/(0.01*Height)^2,
  IndexGroup=cut(Index,breaks = c(0,16,18.5,24.99,30,35,40,60))
  )%>% select(-Date)#,-Mail) %>% filter(Count>1,MRM<RM)
levels(data$CountGroup)=c("2-3","4-6","7-10","11-20",">20")
levels(data$AgeGroup)=c("<20","20-27","28-35",">35")
levels(data$IndexGroup)=c("выраженный дефицит","дефицит","норма","избыток","ожирение1","ожирение2","ожирение3")

ex=data$Experience %>% as.numeric()
ex[ex==6]=5
ex %<>%factor() 
levels(ex)=c("До двух лет","2-3 года","4-5 лет","6-10 лет","больше 10 лет")
data %<>%mutate(Experience=factor(ex,ordered = T)) 

allrows=1:nrow(data)
maxerror=5

#уникальные записи (где один от каждого человека берётся только одна запись)
#объяснить, по каким признакам людей считать одинаковыми
data.unique=data %>% select(AgeGroup,Height,BodyType,Experience,Sex,IndexGroup) %>% unique()

#функции
getparam=function(vec){
  ln=length(levels(vec))
  x=numeric(ln)
  ns=character(ln)
  for(i in seq(ln)){
    x[i]=sum(vec==levels(vec)[i])/length(vec)
    ns[i]=paste0(levels(vec)[i]," (",round(x[i]*100,2),"%)")
  }
  return(list(x=x,ns=ns))
}

getPIE=function(vec,main=""){
  lst=getparam(vec)
  pie(x=lst$x,labels=lst$ns,main=main)
}
getFan=function(vec,main=""){
  pr=getparam(vec)
  fan.plot(pr$x,labels=pr$ns,main=main)
}



data.backup=data




***
# Введение

## Зачем предсказывать повторные максимумы?

Очень часто при подготовке к соревнованиям по пауэрлифтингу или в процессе любительских тренировок **полезно (а иногда и необходимо) оценить свои текущие возможности, не делая "проходку"**, так как "проходка" оказывает сильное воздействие на нервную систему, её нежелательно делать часто и к ней нужно сначала подготавливаться, затем от неё отдыхать (чтобы суметь мобилизовать больше ресурсов и уменьшить риск травм), что занимает много времени и сил --- и всё ради того, чтобы просто узнать, какой именно вес ты способен пожать, присесть или потянуть конкретно сейчас. Гораздо разумнее было бы оценить эти величины (*повторные максимумы*, **ПМ**^[иногда их называют *разовыми максимумами*, **РМ**]), основываясь на результатах, которые ты показывал совсем недавно в процессе обычных тренировок или которые намного проще продемонстрировать^[имеется ввиду, что нетрудно и более безопасно на следующей тренировке взять достаточно большой вес и сделать с ними максимальное число повторений (около 4-8)] (*многоповторные максимумы*, **МПМ**). Кроме того, *если удастся построить такую модель, способную оценить силу человека, исходя из его последних достижений, можно будет сделать более явной связь между тренировками и реальными результатами, предотвращая перетренированность и временные потери на тренировки по программам, не дающим эффекта*.

Итак. **Зачем нужно предсказывать повторные максимумы?** Регрессионные модели, которые требуется построить, помогут:

1. адекватно оценить свои возможности перед соревнованиями и заказать веса, очень близкие к реальному максимуму; кроме того, **знать свой ПМ, не делая проходку, очень важно не только для тех, кто занимается пауэрлифтингом** (см. [статью](https://body1.ru/kalkulyator-odnopovtornogo-maksimuma-1pm/));

1. в период подготовки **оценивать скорость своего прогресса (или вообще его наличие), предотвращая перетренированность**; сравнивать свои результаты с результатами предыдущих циклов, даже если тогда использовалось другое число повторений;

1. **более обоснованно планировать программу тренировок**: пытаясь чередовать разные упражнения и разные диапазоны повторений, легко сделать программу, которая на самом деле не будет давать значимого эффекта или потребует неподъёмных усилий;

1. обосновать действенность разных методик и выразить различия между ними (этому посвящён последний раздел статьи); 

1. более точно замерять силовые качества групп мышц-антагонистов (очень редко для этого вообще возможно измерять 1ПМ у обеих групп, как и МПМ при одном и том же числе повторений);

1. выявить математические закономерности в силовых показателях человека.

## О калькуляторах

К сожалению, многие люди отнеслись к этому исследованию очень скептически, поскольку уверены, что здесь имеется слишком сложная зависимость, чтобы её можно было описывать^[зачастую так думают, люди, которые своему опыту доверяют намного больше, чем научному знаю, тем более в спортивном мире часто всплывают разные псевдонаучные методики, которые приносят спортсменам только вред]; другие утверждали, что знание такой информации принесёт больше вреда, чем пользы, если спортсмен не имеет большого опыта или не тренируется с тренером. Во многом они правы, но на самом деле, как я считаю, **такое мнение обусловлено достаточно большим количеством разных калькуляторов и формул в Интернете, которые дают разные результаты и не ясно на чём основываются** (на каких формулах либо откуда полученных). 

Примеры: 

* формула из книги Брендона Лилли, с которой началось исследование. За более чем месяц я так и не получил ответа от руководства сайта [Juggernaut](https://www.jtsstrength.com), откуда взялась эта формула (чуть позже я выяснил, что это округлённая **формула Вендлера**);

* калькулятор на (весьма хорошем) сайте [Symmetric Strength](https://symmetricstrength.com), аналогичная ситуация;

* формулы [Мориса и Райдина](https://power-fitness.ru/metod-morisa-i-rajdina-ili-kak-uznat-svoj-maksimum-v-zhime-lezha.html), которые, как позже будет показано, нерабочие и, скорее всего, получены из слишком маленьких выборок;

* формулы [О Коннора, Бжицки (Бржыки), Лэндера](http://frs24.ru/st/maksimum-v-zhime-lezha-raschet/) и [другие](https://ru.wikipedia.org/wiki/Одно_повторение_с_максимальным_весом);

* также [этот калькулятор](https://powerliftingrating.ru/repeat_calculator), [этот](http://evgeniyfit.ru/fitnes-kalkulyatoryi/odnopovtornyiy-maksimum-online/), [этот](https://body1.ru/kalkulyator-odnopovtornogo-maksimuma-1pm/) и разные клоны одних и тех же калькуляторов.

По-видимому, многие люди пытались составить формулы для вычисления повторного максимума, но теперь, помимо самих формул, об этих исследованиях мало что известно. 

**Цель же этой статьи -- предоставить формулу, полученную в согласовании с концепциями машинного обучения, которая будет хорошо работать на многих людях, а не на десяти**. Здесь написано, как она получена и почему.


## О данных и ресурсах
Для сбора наблюдений был создан [опрос в Google Forms](https://forms.gle/R4zZqQJ3ggNdEuQ67) на русском языке и [его англоязычный аналог](https://forms.gle/dbthrvE5Y95beqUL9). 

Русскоязычный опрос распространялся через соц. сеть ВКонтакте преимущественно^["преимущественно" означает, что предположительно именно из этих группы поступила основная масса опрошенных] в следующих группах: [Пауэрлифтинг | Тяжелая атлетика](https://vk.com/powerliftingnews), [Я ♥ ПАУЭРЛИФТИНГ](https://vk.com/ilovepowerlifting), [ПАУЭРЛИФТИНГ И ЖИМ ЛЁЖА 18+](https://vk.com/powerliftingworld), [Твой Тренер](https://vk.com/tvoytrenercom); аналогично англоязычный опрос распространялся на Facebook преимущественно здесь: [Powerlifting Motivation](https://www.facebook.com/PowerliftingMotivation/), [International Powerlifting League (IPL)](https://www.facebook.com/groups/IPLPowerlifting/), [Powerlifting Motivation Chat](https://www.facebook.com/groups/powerliftingmotivation/), [Powerlifting Memes](https://www.facebook.com/groups/powerliftingmemes/). Кроме того, несколько наблюдений я собрал лично.

Спустя два месяца от создания опросов данные были выгружены и началась их обработка. Теперь эти данные и всё, связанное с ними (в том числе последняя версия этого документа), хранятся [в моём репозитории](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/tree/master/Estimating%20RM).

Для обработки данных и создания отчёта использовалась среда [RStudio](https://rstudio.com) и **язык R версии `r getRversion()`**. 

## Отправная точка
В книге ["Система тренировок КУБ"](https://forum.steelfactor.ru/index.php?showtopic=44596) на странице 23 (34 в оригинальной версии) приводится формула для определения повторного максимума: 
                    $$предполагаемый\ повторный\ максимум = вес * (1 + 0.0333 * повторения),$$ 
                    
где берётся некоторый рабочий вес и максимально возможное число повторений с ним. В оригинальных обозначениях:

$$RM=WEIGHT \cdot (1+0.0333\cdot REPS).$$

К примеру, если вы можете пожать 100 кг на 5 раз, то из этой формулы следует, что вы сможете пожать 115 на раз и 90 на 9 раз (звучит правдоподобно).

К слову, примерно такая модель с некоторыми поправками на диапазон повторений используется на сайте [Symmetric Strength](https://symmetricstrength.com). Там максимально возможное количество повторений равно 10. 

Лично на мне эта формула хорошо работает и мне захотелось уточнить её для других людей разной комплекции и уровня подготовки. Также интересно то, что **здесь имеется, по сути, очень простая линейная модель с двумя *предикторами* **: рабочим весом и *взаимодействием* рабочего веса с числом повторений, и нет зависимости от самого упражнения, опыта человека и других характеристик, то есть **в перспективе возможно с большой точностью описать наши возможности одной простой формулой, и это будет верно почти для всех людей, невзирая на все различия между ними**.

## Гипотеза и допущения

**Сила человека в одном повторении зависит от пяти составляющих**:

1. **Количество миофибрилл в мышечном волокне**. Чем их больше, тем б**о**льшую силу может развить волокно.

1. **Запасы АТФ и креатинфосфата и уровень ферментации, поддерживающий быстрое преобразование креатинфосфата в АТФ**. Если эти запасы малы, спортсмен не сможет поддерживать максимальное усилие нужные несколько секунд. Сам по себе уровень ферментации для силовых нагрузок можно не учитывать, поскольку после нескольких минут хорошей разминки он становится достаточно высоким.

1. **Соотношение красных и белых мышечных волокон в работающей мышце**: чем больше белых волокон, тем б**о**льшую силу способна развить мышца на короткий промежуток времени. Это соотношение является разным для разных мышц и разных людей (определяется генетикой) и, как пока считается, плохо поддаётся коррекции[^5].

1. **Количество вовлечённых в работу волокон**. Чем их больше, тем больше сила мышцы. Количество задействованных волокон зависит от *импульса ЦНС* и *порога действия органов Гольджи* : чем сильнее импульс, тем больше волокон может вовлечься в работу, однако импульсы выше некоторого порога подавляются органами Гольджи, чтобы человек не мог сознательно применить силу, способную привести к разрыву сухожилий. Тренировки с субмаксимальными и запредельными весами способны воздействовать как на способность генерировать импульс, так и на порог его подавления^[речь идёт о тренировках с более чем 90% от ПМ или с более чем 100% от ПМ в частичной амплитуде, а также об изометрических упражнениях; во многом такие упражнения выполняются для того, чтобы приучить организм поднимать огромные веса, что и значит -- тренировка способности генерировать б**о**льший импульс и увеличение порога его подавления].

1. **Естественные рычаги человека и техника выполнения упражнения**. У каждого человека есть свои рычаги, дающие ему преимущества в тех или иных типах движений. Кроме того, можно откорректировать технику выполнения упражнения так, что амплитуда движения уменьшится в несколько раз, вдобавок при движении основную работу будут выполнять наиболее сильные пучки мышц. Этот фактор не относится конкретно к силе, но фактически его учёт поможет увеличить демонстрируемые результаты.

[^5]: если будет доказано наличие гиперплазии (деления мышечных волокон) в скелетных мышцах человека, можно будет утверждать, что силовые тренировки увеличивают количество белых волокон в работающих мышцах

Подытожив, можно сказать, что **сила спортсмена зависит от генетики (в плане соотношения мышечных волокон, рычагов и т. п.), опыта тренировок (чем больше опыт, тем больше может быть как импульс ЦНС, так и порог его подавления, так и запасы креатинфосфата и т. п.) и конституции (типа телосложения, роста как такового, соотношения роста и функционального веса)**. Сама техника упражнений учитываться не будет.

**Гипотеза исследования** состоит в том, что одноповторный максимум можно с небольшими ошибками предсказать через многоповторный при помощи модели вида

$$RM=MRM \cdot (x+y\cdot REPS)+f(MRM,HEIGHT,WEIGHT)+\varepsilon$$
с поправками на телосложение, опыт тренировок, диапазон повторений или некоторые другие факторы. 

Здесь $x, y$ -- некоторые числа, $f$ -- функция, которую ещё придётся подобрать, $\varepsilon$ -- ошибка, вносимая неучтёнными факторами. При этом предполагается, что искомая зависимость очень близка к линейной, то есть первое слагаемое вносит основной вклад в сумму; это объясняется тем, что интуитивно ясно логическое заключение: из $50*6 \approx 60$ (вес 50 на 6 раз примерно значит 60 на раз) должно следовать $100*6 \approx 120$ с, возможно, небольшими поправками (до 10%) на другие факторы (функция $f$). С другой стороны, формула должна быть верна независимо от того, указывается вес в килограммах или фунтах.

**Поправка на диапазон повторений имеет следующие соображения**. Ввиду физиологии **в разных диапазонах повторений проявляются разные мышечные способности, не все из которых тесно связаны с целевыми мышечными качествами -- абсолютной силой и мощностью**. Почти очевидно, что сила человека в одном повторении хорошо коррелирует с силой в двух-пяти повторениях, но **то, поднимет ли человек указанный вес на 12 раз или на 20, зависит от его силовой выносливости, которая не вносит большого вклада при работе на одно повторение**. Если не вдаваться в подробности, это объясняется тем, что в малом числе повторений основную роль играют запасы креатинфосфата и АТФ, ферментация, напряжение нервного импульса, количество задействованных белых волокон и их сечение, а в большом числе повторений (больше 10-15) существенную роль играет тренированность красных волокон и способность мышц быстро утилизировать продукты метаболических реакций^[в некотором смысле это одно и то же] (если утилизация быстрая, спортсмен сможет сделать на 3-5 повторений больше, а если медленная, то даже огромная сила воли не поможет сделать 2-3 лишних повторения из-за чувствительности к падению pH).

Согласно Хетфилду, сила развивается исключительно при работе с весами не меньше 80% от ПМ и при этом спортсмены среднего уровня способны выполнить 10-15 повторений с 80% от своего ПМ; **то, какой вес человек поднимает на более чем (предположительно) 15 раз, мало говорит о том, какой вес он способен взять на раз**.  

Кроме того, [в статье Вадима Протасенко](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/blob/master/Материалы%20по%20пауэрлифтингу%20и%20не%20только/Тренинг/Вадим%20Протасенко.%20Супертренинг%20без%20заблуждений.pdf) прослеживается такая идея: **разное число повторений требует своего времени на выполнение, а в зависимости от времени работы мышцы включается свой режим энергообмена**: примерно через 7 секунд работы запасы креатинфосфата израсходованы больше чем наполовину, алаклатный режим работы (за счёт креатина) завершается через 12 секунд, потом начинается гликолиз (за счёт гликогена), а через 30-60 секунд работы начинается окисление. При этом, если учитывать, что за 30 секунд обычно делается 10 повторений в жиме и 8 в приседе/тяге, можно сделать выводы, что чисто силовые качества проявляют себя (конечно, приблизительно) в первых пяти повторениях, а силовая выносливость заканчивает оказывать влияние на 15-м повторении; всё, что выше 15-20-ти повторений -- вообще не имеет отношения к силе[^6].

[^6]: конечно, это лишь условные размышления без учёта особенностей отдельных частей тела и того, что ввиду усталости каждое следующее повторение делается дольше предыдущего. Важно лишь усвоить, что разные диапазоны повторений по-разному связаны с силой.

<p align="center">
  <img src="Время подхода.png">
</p>

## Соображения о возможности построения модели

К сожалению, планируемая **модель априори не сможет идеально точно предсказывать результаты конкретных спортсменов**. Существует также **множество причин, почему модель, возможно, не выявит реальные закономерности в генеральной совокупности**:

* Опрос проводился почти полностью в Интернете, точность и честность ответов остаются на совести испытуемых. Быть может, кто-то грубо округлил свои данные или назвал данные, которые нельзя считать идентичными (например, разница между выявлением ПМ и МПМ составляла большой срок, отчего уже нельзя говорить, что исследуемые возможности принадлежат одному и тому же *телу* ) либо не совсем понял целей опроса и назвал заниженные результаты (например, тяжёлый разминочный подход на 3 повторения, когда в действительности возможно сделать 5).

* Через опрос невозможно выявить, предположительно, такие имеющие большое значение факторы как естественные рычаги, процент жира в организме (процент веса, который *условно* не имеет вклада в результаты, зато может испортить статистику), особенности техники и качества выполнения движений. По этой причине многие люди, которые в рамках опроса имеют идентичные параметры, могут иметь совершенно разные результаты, поскольку при одинаковом росте, весе и возрасте у одного больше функциональной массы, у другого ноги длиннее, один использовал бинты (дающие прибавку 5-15 кг и не считающиеся экипировкой), другой нет, один жал по всем правилам, другой в отбив и т. п.

* Есть множество факторов, оказывающих большое влияние, трудно измеримых и при этом варьирующихся у одного и того же человека. Например, при одних и тех же мышцах человек может поднять один и тот же вес на 3 или 5 раз в зависимости от уровня креатинфосфата и психологического настроя; при следующей тренировке эти параметры могут быть уже значимо другими, отчего правила, хорошо работавшие недавно, перестанут хорошо работать сейчас.

В то же время имеются **доводы в пользу модели**:

* ПМ предсказывается не только на основе роста или веса, но и на основе многоповторного максимума, то есть не многим меньшего веса и числа повторений с ним. Нельзя отрицать, что при каких-то условиях МПМ всё же содержит немалую информацию о ПМ.

* Существует модель Вендлера (и схожие подели), которая приемлемо работает для многих спортсменов. Сам факт наличия такой модели говорит о том, что ПМ можно описать через МПМ в какой-то степени или хотя бы для какой-то группы людей. 

**Можно долго рассуждать о том, будет работать модель или нет и почему, но для того и проводится исследование, чтобы найти математически обоснованный ответ**.

## План исследования

Сперва будет произведён **разведочных анализ собранных данных** с целью выявить в них некоторые закономерности, подтвердить или опровергнуть некоторые простые гипотезы, обнаружить выбросы, принять решение о включении тех или иных факторов в модель.

После этого **будет построена серия моделей, среди которых будет выбрана лучшая**. По правилам машинного обучения лучшей моделью считается та, что даёт наименьшую ошибку при перекрёстной проверке и (согласно принципу бритвы Оккама) наиболее проста в объяснении. Параллельно будем проверять достоверность уже известных моделей.

В конце я сделаю выводы и опишу возможности применения полученной модели.

***
# Описание выборки и разведочный анализ данных

## О прошедших опрос
С помощью опроса было получено **всего `r nrow(data)` достоверных наблюдения** (исключая явные аномалии), принадлежащие предположительно **`r nrow(data.unique)`** людям[^10]; они содержатся в файле [data(rus).tsv](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/blob/master/Estimating%20RM/data(rus).tsv).

Все наблюдения содержат информацию о нескольких переменных. После некоторых преобразований над этими переменными получаем следующий набор предикторов:

[^10]: дело в том, что одни и те же люди могли вносить более одного ответа. В итоге два наблюдения считаются принадлежащими одному человеку, если они совпадают по переменным **AgeGroup**, **Height**, **BodyType**, **Experience**, **Sex**, **IndexGroup** (которые вряд ли изменятся за время между фиксацией разных наблюдений)

* **RM** -- собственно повторный максимум

* **MRM** -- многоповторный максимум

* **Count** -- количество повторений для многоповторного максимума

* **Action** -- движение для которого верны измерения (`r data$Action %>% levels()` -- самые базовые упражнения)

* **Sex** -- пол испытуемого

* **Experience** -- группа опыта (`r data$Experience %>% levels()`)

* **Age** -- возраст

* **Weight** -- собственный вес

* **Height** -- рост

* **BodyType** -- тип телосложения (`r data$BodyType %>% levels()`). Типы телосложения в целом различаются скоростью обмена веществ и строением скелета, рычагами (хотя различия в строении скелета во многом являются следствием скорости обмена веществ) 

* **CountGroup** -- группа по диапазону повторений (`r data$CountGroup %>% levels()`)

* **AgeGroup** -- возрастная группа (`r data$AgeGroup %>% levels()`)

* **Index** -- [индекс массы тела](https://ru.wikipedia.org/wiki/Индекс_массы_тела), **ИМТ**

* **IndexGroup** -- группа по индексу массы тела (`r data$IndexGroup %>% levels()` в соответствии с рекомендациями ВОЗ)

Основные статистики по данным переменным:

In [ ]:
numcols=sapply(data,is.numeric)

#для числовых переменных
tt=data[,numcols]%>% psych::describe() 
tt[,c(3,4,5,7,8,9,10)]%>% kable(digits=3)

#для факторных переменных
data[,!numcols] %>% select(-Mail) %>% 
  summary() %>% 
  xtable(caption='Количество наблюдений в каждой группе по факторным переменным')


Сама **таблица data** (используемый датасет), отсортированная по некоторым столбцам:

In [ ]:
data %>% select(-Mail)

**Сделаем некоторые выводы о выборке**. Почти все испытуемые -- мужчины, поэтому *результаты, которые будут получены, не следует обобщать на женщин*. Возможно даже, что позднее придётся удалить принадлежащие женщинам наблюдения из выборки, если окажется, что те сильно выделяются.

In [ ]:
getPIE(data.unique$Sex,main = "Пол испытуемых")

Среди испытуемый почти половину составляли эндоморфы:

In [ ]:
getPIE(data.unique$BodyType,main="Тип телосложения испытуемых")

и тест хи-квадрат говорит, что это отличие статистически значимо, то есть **среди пауэрлифтеров эндоморфы встречаются чаще эктоморфов или мезоморфов**:

In [ ]:
chisq.test(data.unique%>% filter(Sex=="Мужчина") %>%select(BodyType) %>% table()) %>% pander::pander()

Больше половины наблюдений относятся к жиму лёжа. Я думаю, это связано с тем, что большинство спортсменов просто предпочитают это упражнение двум другим, вдобавок на жиме лёжа относительно проще замерять МПМ и, скорее всего, многие из ответивших специализировались именно на жиме лёжа (жимовики).

In [ ]:
getPIE(data$Action, main="Движение")

**Распределение по другим факторам**:

In [ ]:
par(mfrow=c(1,1),mai=rep(0.7,4))
getPIE(data$CountGroup,main = "Диапазон повторений")
getPIE(data$Experience,main = "Опыт тренировок")
getPIE(data$AgeGroup,main = "Возрастная группа")
#getPIE(data$IndexGroup,main = "Группа по индексу массы тела")
par(mfrow=c(1,1))

ggplot(data,aes(x=CountGroup,fill=Action))+geom_bar()+theme_classic()+labs(title="Количество наблюдений в каждом диапазоне повторений",x="Диапазон повторений",y="Количество",fill="Движение")

ggplot(data,aes(x=Experience,fill=AgeGroup))+geom_bar()+theme_classic()+labs(title="Распределение по опыту тренировок",x="Опыт тренировок",y="Количество",fill="Возраст")

(ggplot(data,aes(x=AgeGroup,fill=IndexGroup))+geom_bar()+theme_classic()+labs(title="Распределение по возрасту",x="Возрастная группа",y="Количество",fill="ИМТ")) %>% ggplotly()

**Распределение по индексу массы тела**:

In [ ]:
#среди эндоморфов избыток веса встречается почаще
data.unique %$%  table(BodyType,IndexGroup) %>% kable()

ggplot(data.unique,aes(x=BodyType,fill=IndexGroup))+geom_bar(position=position_dodge2())+
  labs(x="Тип телосложения",y="Количество ответивших",fill="Индекс массы тела")+
  coord_flip()+theme_bw()+theme(legend.position = "bottom")

Видно, что среди ответивших нет людей с дефицитом веса и немалая часть имеет нормальный вес, немалая -- "избыток" (предположительно за счёт мышечной массы), но так называемое **"ожирение1" (с большой вероятностью связанное с жировыми отложениями) у эндоморфов встречается чаще, чем в других телосложениях**. Это подтверждает тест пропорций:

In [ ]:
tb=data.unique%>% 
  mutate(Obees=ifelse(IndexGroup=="ожирение1","да","нет")) %>%
  select(Obees,BodyType) %>% table()

tb %>% t() %>% prop.test() %>% pander()#для всех

data.unique%>% 
  mutate(Obees=ifelse(IndexGroup=="ожирение1","да","нет"),
         Bd=ifelse(BodyType=='Эндоморф',1,0)) %>%
  select(Obees,Bd) %>% 
  table() %>% t() %>% 
  prop.test()%>% pander() # eсли разделить на эндоморфов и всех остальных

Аналогично дисперсионный анализ показывает, что **эндоморфы имеют больший ИМТ**:

In [ ]:
aov(Index~BodyType,data) %>% summary() %>% pander()

t.test(data %>% filter(BodyType!='Эндоморф') %>% select(Index) %>%t() %>% as.numeric(),
       data %>% filter(BodyType=='Эндоморф') %>% select(Index)%>%t() %>% as.numeric()) %>% pander()

## Взаимодествия переменных

Иллюстрация некоторых парных взаимодействий:

In [ ]:
GGally::ggpairs(data%>% select(RM,MRM,Action,BodyType),
                title="Диаграммы взаимодействий между переменными в выборке",
                lower = list(combo = "box")) #%>% ggplotly()

**Корреляции количественных переменных**:

In [ ]:
GGally::ggcorr(data,label=T,label_round = 2)

Зависимость повторного максимума от индекса массы тела:

In [ ]:
ggplot(data,aes(x=IndexGroup,y=RM))+geom_boxplot()+facet_wrap(vars(Action))+coord_flip()+
  labs(x="Группа по индексу массы",y="Повторный максимум",
       title="Зависимость повторного максимума от индекса массы тела",
       subtitle = "Из графика видно, что жим лёжа имеет тенденцию увеличиваться\n с ростом индекса массы тела. \nОднако для приседа и тяги это верно лишь до некоторого порога")+
  theme_bw()
  
#ggplot(data,aes(x=IndexGroup,y=RM))+geom_boxplot()+facet_grid(Action~BodyType)

ggplot(data,aes(x=Index,y=RM,col=BodyType))+geom_point()+
  facet_wrap(~Action)+geom_smooth(method = "lm",se = F)+
  theme_bw()+theme(legend.position = "bottom")+
  labs(x="Индекс массы тела",y="Повторный максимум",col="Телосложение",
       title = "Зависимость повторного максимума от индекса массы тела")

Оценка коэффициента простой регрессионной модели в зависимости от числа повторений:

In [ ]:
data %>% ggplot(aes(x=factor(Count),y=(RM/MRM-1)/Count))+
  geom_boxplot()+theme_bw()+
  geom_hline(yintercept = 0.0333,size =1.3,col="red")+
  labs(x="Число повторений",title = "Оценка параметра для разного числа повторений",
       subtitle = "Красным цветом обозначен параметр 0.0333 из книги Лилли. \nКак видно, он может быть верен для числа повторений от 2 до 5")


Из следующего графика может следовать, что начиная с 8-9 повторений зависимость между МПМ и ПМ ослабевает:

In [ ]:
data.ct=data %>% filter(Count<=10|Count==12)

#По этому соотношению надо бы выбросы удалить
prc=data.ct %>% ggplot(aes(x=factor(Count),y=100*MRM/RM))+geom_boxplot()+coord_flip()+theme_bw()+
  labs(x="Число повторений",y="Какой процент составляет МПМ от ПМ")
prc #%>% ggplotly() %>% I()

Либо здесь имеется проблема с данными, так как на 9-м повторении корреляция резко уменьшается (мало наблюдений):

In [ ]:

tibble('Число повторений'=2:10,
       'Корреляция MRM & RM'=sapply(2:10, function(x) data %>% filter(Count==x) %$% cor(RM,MRM)),
       'Нижняя граница'=sapply(2:10, function(x) data %>% filter(Count==x) %$% cor.test(RM,MRM)$conf.int[1]),
       'Верхняя граница'=sapply(2:10, function(x) data %>% filter(Count==x) %$% cor.test(RM,MRM)$conf.int[2]),
       'Всего наблюдений'=sapply(2:10, function(x) data %>% filter(Count==x) %>% nrow()))%>% 
  kable(caption = "Корреляция между MRM & RM для каждого повторения",align = 'ccccc')


**Есть ли разница в этих процентах для разных движений или телосложений? Дисперсионный анализ показывает, что нет**, значимых различий не обнаружено (все p-значения больше 0.05):

In [ ]:
#есть ли разница в проценте в зависимости от чего-то

#cat("p-значения для телосложений:\n")
pvalues1=sapply(2:10, function(x) data %>% filter(Count==x) %$% aov(MRM/RM~BodyType,.)%>% summary() %$% .[[1]][["Pr(>F)"]][1])
names(pvalues1)=paste(2:10,"repeats")


#cat("p-значения для типа движения:\n")
pvalues2=sapply(2:6, function(x) data %>% filter(Count==x) %$% aov(MRM/RM~Action,.)%>% summary() %$% .[[1]][["Pr(>F)"]][1]) %>% round(7)
names(pvalues2)=paste(2:6,"repeats")


#cat("p-значения для групп по индексу массы:\n")
pvalues3=sapply(2:10, function(x) data %>% filter(Count==x) %$% aov(MRM/RM~IndexGroup,.)%>% summary() %$% .[[1]][["Pr(>F)"]][1])
names(pvalues3)=paste(2:10,"repeats")


tibble('Число повторений'=2:10,'p-значения для телосложений'=pvalues1,'p-значения для типа движения'=c(pvalues2,rep('мало данных',4)),'p-значения для групп по индексу массы'=pvalues3)%>% kable(caption = "p-значения для разных факторных переменных на каждом числе повторений",align = 'cccc')


Применим **тест Стьюдента для определения предсказываемых процентов и доверительных интервалов для них**:

In [ ]:
df=data %>% mutate(perc=100*MRM/RM) %>% filter((Count<=12&Count%%2==0)|Count==15|Count==20) %>% 
  group_by(factor(Count)) %>% 
  summarise(mean=t.test(perc,conf.level = 0.99)$estimate,
            down=t.test(perc,conf.level = 0.99)$conf.int[1],
            up=t.test(perc,conf.level = 0.99)$conf.int[2]) 
names(df)=c("Число повторений","Ожидаемый %","Нижняя граница","Верхняя граница")

df %>% kable(caption="Какой % составляет МПМ от ПМ",align = 'ccc')

В целом эти данные согласуются с тем, что используются [National Strength and Conditioning Association (NSCA)](https://www.nsca.com):

<p align="center">
  <img src="NSCA2.jpg">
</p>

Хотя есть и отличия: вес на 2 повторения скорее равен 94% от максимума, а не 95%, а вес на 12 повторений -- это скорее 70% от максимума, а не 67%.

Теперь посмотрим на зависимость повторного максимума от многоповторного:

In [ ]:
plt=data %>% ggplot(aes(x=MRM,y=RM))+geom_smooth()+
  geom_point(aes(col=CountGroup),size=3)+theme_bw()+
  #facet_grid(vars(CountGroup))+
  labs(x="Многоповторный максимум",y="Повторный максимум",
       col="Диапазон повторений",
       title = "Зависимость повторного максимума от многоповторного")
plt+theme(legend.position = c(0.85,0.2))

Очевидно, что здесь будет близкая к линейной зависимость (учитывая высокую корреляцию).
Оказывается, если предварительно разбить наблюдения на группы по повторениями, линейность станет намного более выраженной для числа повторений не больше 10 (для большего числа повторений не прослеживается явной линейности либо из-за недостатка данных, либо по физиологическим причинам, озвученным ранее; для диапазона 11-20 возможна квадратичная зависимость, но мало данных, чтобы что-то подтвердить):

In [ ]:
plt+facet_grid(vars(CountGroup))+theme(legend.position ='bottom')

Сказанное выше значит, что для каждого диапазона повторений нужен отдельный анализ.

Есть ли зависимость между отношением $\frac{RM}{MRM}$ в зависимости от опыта тренировок и других факторов? Дисперсионный анализ показывает, что в целом нет:

In [ ]:
#есть ли значимые различия в разных возрастных группах для фиксированного диапазона

tibble(
  'Диапозон повторений'=levels(data$CountGroup),
  'Опыт'=sapply(levels(data$CountGroup),function(x) aov(RM/MRM~Experience,data %>% filter(CountGroup==x))%>% summary()%$% .[[1]][["Pr(>F)"]][1]),
       'Возраст'=sapply(levels(data$CountGroup),function(x) aov(RM/MRM~AgeGroup,data %>% filter(CountGroup==x))%>% summary()%$% .[[1]][["Pr(>F)"]][1]),
       'ИМТ'=sapply(levels(data$CountGroup),function(x) aov(RM/MRM~IndexGroup,data %>% filter(CountGroup==x))%>% summary()%$% .[[1]][["Pr(>F)"]][1]),
       'Телосложение'=sapply(levels(data$CountGroup),function(x) aov(RM/MRM~BodyType,data %>% filter(CountGroup==x))%>% summary()%$% .[[1]][["Pr(>F)"]][1])) %>% 
  kable(caption = 'Значимость различий отношения ПМ к МПМ в каждом диапазоне повторений при разных уровнях факторных переменных',align = 'ccccc')


Единственное: обнаружилась разница в зависимости от возраста и индекса массы тела для диапазона повторений 11-20. Возможно, это связано с тем, что в целом с увеличением возраста увеличивается уровень подготовки, отчего на более чем 10 раз удаётся поднимать больший процент от максимального веса.

In [ ]:
ggplot(data,aes(x=AgeGroup,y=RM/MRM))+geom_boxplot()+
  facet_grid(vars(CountGroup))+theme_bw()+
  labs(x="Возраст")

Кроме того, **возможна разница между самими упражнениями** (движениями):

In [ ]:
# p-значения в зависимости от типа телосложения

(sapply(levels(data$CountGroup),function(x) aov(RM/MRM~Action,data %>% filter(CountGroup==x))%>% summary()%$% .[[1]][["Pr(>F)"]][1])) %>% pander()

ggplot(data %>% filter(Count<=20),aes(x=Action,y=RM/MRM))+geom_boxplot()+
  facet_grid(vars(CountGroup))+theme_bw()+
  labs(x="Движение")+coord_flip()

Дополнительные закономерности:

In [ ]:
bx=ggplot(data)+geom_boxplot(aes(x=CountGroup,y=RM/MRM))+
  labs(title = "Отношение повторного максимума к многоповторному \nв зависимости от числа повторений",
       x="Диапазон повторений",y="Отношение повторного максимума к многоповторному",
       caption = "Каждый диапазон повторений действует по своим законам \n и нуждается в собственной версии модели")+
  theme_tq()
bx

bx+facet_grid(~BodyType)+theme_bw()+labs(caption="Для числа повторений до 10 тип телосложения не имеет видимого значения. \nНа высоком числе повторений эндоморфы менее выносливы")
#bx+facet_grid(~Sex)+theme_bw()+labs(caption="caption")

bx+facet_grid(~Action)+theme_bw()+labs(caption="Присед имеет хорошую выносливость на большом числе повторений, жим -- на маленьком",
 subtitle = "Чем отношение ниже, тем более 'выносливы' мышцы в том или ином диапазоне")


## Исправление опечаток

Часто при сборе данных возникают *ненормальные* наблюдения, полученные из-за опечаток или недопониманий. В этот раз таких наблюдений накопилось около 10%. Если поразмыслить над их причинностью, то чаще всего испытуемый забывал поставить цифру "1" перед числом, в результате вместо роста 175 получался рост 75 и т. п. 

В то же время могло быть и так, что испытуемый неверно понял суть эксперимента и в анкете описывал просто подход, который был выполнен, а не подход для многоповторного максимума (то есть на пределе усилий), либо при проходке использовал экипировку или другую технику упражнения, либо, опять же, опечатался; в итоге жим 170х5 оказывался равным 220, хотя ясно, что тут должны стоять 170х5 и 200 либо 170х15 и 220.

С такими *выбросами* можно сделать только две вещи:

1. исправить их, опираясь на логику либо переспросить того, кому это наблюдение принадлежит^[к сожалению, этого не получилось сделать, так как испытуемые редко пользуются электронной почтой, видимо];
1. удалить из выборки.

В итоге некоторые данные были исправлены на более вероятный вариант, а явные непонятные аномалии были удалены из выборки. [В файле с исходными данными](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/blob/master/Estimating%20RM/data.tsv) содержатся комментарии, где и что я исправил и почему.

## Удалённые данные

Перед построением модели из обучающей выборки заранее **были исключены следующие наблюдения**:

- некоторые **выбросы**, в том числе несколько наблюдений, которые изначально не вписывались в модель и по логике очень сомнительны; речь идёт о наблюдениях типа $120*3=140$, хотя интуитивно понятно, что 140 -- это как минимум 120 на 5; возможно, подход 120 на 3 был сделан в достаточно отдалённый от проходки период либо во время проходки в качестве разминочного, то есть его нельзя считать многоповторным максимумом, эквивалентным повторному максимуму; конечно, здесь может играть роль какой-то особый случай[^4], который является выбросом (потому должен быть исключён) либо не может быть выявлен по тому набору признаков, которые доступны в нашем случае;  

- **наблюдения с числом повторений больше 20**, так как сама идея предсказать 1ПМ по, например, 30 ПМ -- абсурдна, поскольку здесь имеют место в корне разные нагрузки, задействуются разные типы мышечных волокон и имеет место разный энергетический обмен; по этой причине использование таких данных при создании моделей приводит к искажениям, а для самих этих наблюдений модель предсказывает лишь какую-то нижнюю границу 1ПМ, в чём нет смысла[^3]; вдобавок, для таких предсказаний сложнее определить аномальные наблюдения в смысле предыдущего пункта; кроме того, даже на графиках видно, что диапазон выше 20 очень сильно отличается от остальных диапазонов.

[^3]: такую нижнюю границу можно предсказать, просто умножив вес примерно на 1.5, исследование же рассчитано на поиск верхней границы

[^4]: например, высокий импульс ЦНС, не соответствующий уровню креатинфосфата. В таком случае человек может на раз пожать большой вес, но на несколько раз уже *недостаточно большой*: при ПМ в 140 на 3 раза жмётся не почти 130, а 120

Как итог, теперь для данных верны следующие статистики:

In [ ]:
data %<>%filter(Count<=20) 

data %>% select(-Mail) %>% summary()

plt=ggplot(data)+theme_bw()

plt+geom_density(aes(x=RM),fill="green")+geom_density(aes(x=MRM,fill="red",alpha=0.5))+
  labs(x="RM (зелёное), MRM (красное)",y="ядерная плотность",title = 'Плотность распределения повторного максимума \nи многоповторного максимума')+
  theme(legend.position = 'none')

plt+geom_bar(aes(x=factor(Count)))+labs(x="Количество повторений",y="Количество наблюдений")+coord_flip()

plt+geom_bar(aes(x=CountGroup,fill=BodyType),position=position_dodge2())+theme(legend.position = c(.85,.8))+
  labs(x="Диапазон повторений",y="Количество", title = "Количество наблюдений в каждом диапазоне повторений",fill="Телосложение")

plt+geom_bar(aes(x=IndexGroup,fill=BodyType),position=position_dodge2())+theme(legend.position = c(.85,.8))+
  labs(x="Категория по индексу массы тела",y="Количество", title = "Количество наблюдений в каждой категории по индексу массы тела",fill="Телосложение")

plt+geom_bar(aes(x=AgeGroup,fill=BodyType),position=position_dodge2())+theme(legend.position = c(.85,.8))+
  labs(x="Категория по возрасту",y="Количество", title = "Количество наблюдений в каждой категории по возрасту",fill="Телосложение")

plt+geom_bar(aes(x=Action,fill=BodyType),position=position_dodge2())+theme(legend.position = c(.85,.8))+
  labs(x="Движение",y="Количество", title = "Количество наблюдений в каждом движении",fill="Телосложение")

plt+geom_point(aes(x=Weight,y=Height,col=AgeGroup,shape=Sex),size=2.5)+
  facet_grid(vars(Action),vars(BodyType))+
  labs(x="Вес",y="Рост",shape="Пол",col="Возраст",title = "Зависимость между ростом и весом")

## Подведение итогов, отбор признаков

Перечислим основные идеи об особенностях данных, сказанные ранее:

* Любые **полученные результаты следует обобщать на женщин**, так как в опросе они почти не участвовали. Возможно, ради точности вычислений даже придётся убрать женщин из выборки;

* **Эндоморфы среди пауэрлифтеров встречаются чаще мезоморфов или эктоморфов. При этом у эндоморфов значимо чаще встречается "ожирение первой стадии", чем у представителей других телосложений**;

* **Результаты в приседе и тяге растут с увеличением индекса массы тела лишь до некоторого порога**. Увеличение индекса массы тела выше "ожирения первой стадии" не будет полезным;

* **Сильная корреляция между индексом массы тела и повторным максимумом наблюдается у мезофорфов -- в жиме, у эктоморфов -- в тяге, у эндоморфов -- в приседе**;

* Не обнаружено значимой разницы значений $\frac{MRM}{RM}$ для разных телосложений, движений, групп по индексу массы, опыта, возраста для любого конкретного числа повторений или диапазона повторений (но при этом обнаружилась разница между движениями для диапазонов повторений до 10); **процентовки от *National Strength and Conditioning Association (NSCA)* с небольшими погрешностями верны для всех спортсменов и всех движений**;

* **Разбиение наблюдений на группы по диапазону повторений (который сделан согласовано с физиологическими соображениями) должно значительно повысить точность модели**, причем для числа повторений до 10-20 эта модель крайне близка к линейной;

* Наблюдения свыше 20 повторений не будут учитываться, так как их мало, вдобавок этот диапазон слабо коррелирует с абсолютной силой;

* **При более чем 10 повторений опытные спортсмены способны поднимать более близкие к максимальным веса, чем неопытные**.

Соображения о предикторах для модели:

* $RM$ (повторный максимум) обязательно зависит как от $MRM$ (многоповторный максимум) и $Count$ (числа повторений), так и от $CountGroup$ -- группы по числу повторений;

* $RM$ может зависеть также от $BodyType$ (типа телосложения) и $Action$ (упражнения) либо от их бинарных модификаций (например, если людей разделить не на 3 телосложения, а на эндоморфов и нет); возможно, стоит также попробовать учесть индекс массы тела или собственно вес, имеющие корреляцию с $RM$ не меньше 0.5.

Дальнейшие действия:

1. Просмотреть, как работает модель Вендлера, и на её примере ознакомиться с шаблоном по оценке качества, который будет использоваться для оценки следующих моделей.

1. Построить разные модели для оценки конкретно $RM$ и выбрать ту, которая более проста и даёт меньшую ошибку кросс-валидации.

1. Аналогично построить модели для $\frac{MRM}{RM}$ и выбрать лучшую.

1. Сравнить две выбранные модели и определить лучшую из них.


***
# Построение моделей

In [ ]:
data %<>%filter(Count<=20) 
#data %>% summary()

Error=function(target,weight) (target-weight)^2 %>% mean() %>% sqrt()

Show=function(vals,df=data){
  #vals=predict(model,df)
  
  
  cat('Ошибки модели (выведены 15 элементов с наибольшими ошибками): \n')
  
  err=df$RM-vals
  cbind(Fact=round(vals),Target=df$RM,
        ERROR=abs(df$RM-vals),
        ErrorPercent=abs(err)/df$RM*100,
        df[,c(2:15)]) %>% tbl_df() %>% select(-Age,-Mail,-Experience,-AgeGroup,-IndexGroup,-BodyType,-Height)%>% arrange(-ERROR,-ErrorPercent,Weight) %>% 
        filter(ERROR>1)%>%head(15) %>% kable() %>%  print()
  cat("\n")
  rg=range(err)#;print(err);print(rg)
  
  if(rg[1]<0)cat("------------> Наибольшая ошибка в большую сторону:",-rg[1],"\n")
  if(rg[2]>0)cat("------------> Наибольшая ошибка в меньшую сторону:",rg[2],"\n")
  
  s=sum(abs(err)/df$RM*100>maxerror)
  len=length(err)
  cat("Модель ошиблась более чем на",maxerror,"% в",s,"случаях из",len,"(",s/len*100,"%)\n")
  s=sum(abs(err)>maxerror)
  cat("Модель ошиблась более чем на",maxerror,"кг в",s,"случаях из",len,"(",s/len*100,"%)\n")
  
  cat("----------------> Статистика по ошибкам в процентах:\n")
  (abs(df$RM-vals)/df$RM*100) %>% summary() %>% print()
  cat("-------------------> Среднеквадратичная ошибка:", Error(vals,df$RM),"\n")
}

ShowErrors=function(model,power.coef=1,sum.coef=0){
  Show(predict(model,data)*power.coef+sum.coef)
  
  cat("Оценка кросс-валидации для всего набора данных",
      boot::cv.glm(data,glm(formula = model$call$formula,data=data),K=10)$delta[1],"\n")
  cat("Оценка кросс-валидации для не более чем 10 повторений",
      boot::cv.glm(data %>% filter(Count<11),glm(formula = model$call$formula,data=data %>% filter(Count<11)),K=10)$delta[1],"\n") 
  cat("Оценка кросс-валидации для не более чем 6 повторений",
      boot::cv.glm(data %>% filter(Count<7),glm(formula = model$call$formula,data=data %>% filter(Count<7)),K=10)$delta[1],"\n") 
}


ResAn=function(res){
  
  p=ggplot(data %>% mutate(res=res),aes(x=CountGroup,y=res))+
    geom_boxplot()+labs(x="Группа повторений",y="Остатки (цель - предсказание)",title="Распределения остатков в зависимости от группы повторений")+theme_bw()
  print(p)
  
  (p+facet_grid(vars(Action))) %>% print()
  
  (p+facet_grid(vars(BodyType),vars(Action)))%>% print()
  return(0)
}

#из этого графика можно сделать вывод, что модель неплохо работает на диапазоне 2-3, но на диапазоне 13-20 ошибка какая-то сильно отличающаяся от тенденции уменьшения ошибок, так что этот диапазон надо бы и вообще убрать, так как там уже играют роль свойства красных волокон, не говорящие о силе
#ResAn(data$RM-data$MRM*(1+0.0333*data$Count))

ResVal=function(vals)ResAn(data$RM-vals)

#ResGraf=function(model)ResVal(predict(model,data))

mysummary=function(mdl){
 
  cat("-----> ОБЩАЯ ИНФОРМАЦИЯ О МОДЕЛИ:\n");cat("\n")
 gvlma::gvlma(mdl) %>% summary() %>% print();cat("\n")
 
 
 cat("-----> БАЗОВЫЕ ГРАФИКИ:\n");cat("\n")
 par(mfrow=c(2,2))
 plot(mdl)
 par(mfrow=c(1,1)) ;cat("\n")
 
 
 cat("-----> ТЕСТ НА НОРМАЛЬНОСТЬ РАСПРЕДЕЛЕНИЯ ОСТАТКОВ\n");cat("\n")
 shapiro.test(mdl$residuals) %>% print();cat("\n")
 
 qqPlot(mdl,main="Q-Q plot")  
 
 
 cat("-----> ФАКТОР ИНФЛЯЦИИ ДИСПЕРСИЙ:\n");cat("\n")
 vif(mdl)%>% print();cat("\n")
 
  cat("-----> ТЕСТ НА АВТОКОРРЕЛЯЦИЮ:\n");cat("\n")
  durbinWatsonTest(mdl) %>% print();cat("\n")   #тест на автокорреляцию
  
 # cat("-----> ТЕСТ НА ОДНОРОДНОСТЬ ДИСПЕРСИИ:\n");cat("\n")
  # ncvTest(mdl)%>% print();cat("\n")    #однородность дисперсии

  cat("-----> ТЕСТ НА ВЫБРОСЫ И ВЛИЯТЕЛЬНЫЕ НАБЛЮДЕНИЯ:\n");cat("\n")
  
  outs=outlierTest(mdl)
  outs%>% print()
  
  
  influ=influencePlot(mdl,main="Диаграмма влияния",sub="Размеры кругов пропорциональны расстояниям Кука")
gb=capture.output(influ %>% print())  
  
  cat("-----> ВЫБРОСЫ И ВЛИЯТЕЛЬНЫЕ НАБЛЮДЕНИЯ:\n")
  data[c(outs$p %>% names(),influ %>% rownames()) %>% as.numeric(),] %>% unique() %>% 
  select(-Mail,-Action2,-Body2,-AgeGroup,-CountGroup,-Index) %>% arrange(-RM,-Count) %>% kable() %>% print();cat("\n")
  
}

all=function(modelka){
  modelka %>% ShowErrors()
  
  modelka %>% predict(data) %>% ResVal()
  
  modelka %>% mysummary()
}

data %<>% mutate(Body2=ifelse(BodyType=="Эндоморф","Endo","NonEndo") %>% factor(),
                Action2=ifelse(Action=="Жим","Up","Down") %>% factor())

Вычислительные возможности позволяют создать и проверить огромное количество моделей. Цель исследователя -- предложить несколько вариантов и выбрать среди них лучший. Кроме этого, нужно посмотреть, как работает модель на разных группах данных, чтобы обнаружить выбросы или обнаружить, что на такой-то группе модель вообще не может работать, из-за чего такую группу придётся исключить (эта работа уже проделана при самом исследовании, чтобы не нагружать статью).

Итак, в модели представляют интерес следующие показатели:

* ошибки в разных группах данных;

* выбросы и влиятельные наблюдения;

* статистическая значимость модели;

* оценки кросс-валидации.

## Недостатки исходного решения

Применив формулу $RM=MRM \cdot(1+0.0333 \cdot Count)$ ко всем наблюдениям, получим оценки $RM$ (Fact), которые будут отличаться от истинного $RM$ (Target). Разница между этими величинами -- остатки. Для модели Вендлера остатки будут следующие:

In [ ]:
gb=capture.output(ResVal(data$MRM*(1+0.0333*data$Count))) 

Здесь такая логика: чем лучше модель, тем "ящики" тоньше и тем их середины ближе к 0.

В данном случае видно, что **модель Вендлера хорошо работает почти на всех данных из диапазона 2-3 и на немалой части данных из диапазона 4-6**. Для большего числа повторений модель даёт завышенные оценки.

Если посмотреть на численные ошибки

In [ ]:
Show(data$MRM*(1+0.0333*data$Count))

увидим, что модель заметно ошибается в 30% случаев и в среднем на почти 5%. Первые формулы [Мориса и Райдина](https://power-fitness.ru/metod-morisa-i-rajdina-ili-kak-uznat-svoj-maksimum-v-zhime-lezha.html) показывают худший результат:

In [ ]:
Show(data$MRM*(
  0.969611*exp(0.030583*data$Count)*ifelse(data$Action=="Жим",1,0)+
    0.985993*exp(0.015050*data$Count)*ifelse(data$Action!="Жим",1,0)
))

На самом деле это две формулы вида

$$\log \left(\frac{RM}{MRM}\right)= a+b \cdot Count,$$

где $a,b$ зависят от того, к какой части тела относится упражнение. Сама идея сделать логарифмическое преобразование используется нередко, но здесь это имеет негативный эффект: $R^2$ равен далеко не 0.99, как сказано на сайте:

In [ ]:
lm(log(RM/MRM)~Action2+Count:Action2,data)%>% summary() %>% pander()

Скорее всего, формулы Мориса и Райдина подгонялись на маленькой выборке или на выборке, в которой почти все данные принадлежали нескольким людям.

## Уточнение коэффициентов

Теперь возьмём модель вида  $RM=MRM \cdot(a+b \cdot Count)$ и подберём коэффициенты $a,b$ лучшим образом. В итоге:

In [ ]:
lm(RM~MRM+MRM:Count-1,data) -> b1 
b1 %>% all()

Выше приведена вся необходимая информация о качестве модели (назовём эту модель $b_1$). Из неё важнее всего следующее:

* На выборочных данных модель ошибается максимум на 21кг и 13%;

* В среднем модель ошибается на 3%;

* В 75% случаев ошибка не превышала 5%;

* Наибольший разброс ошибок приходится на диапазон 11-20;

* Модель статистически значима и удовлетворяет всем нужным требованиям, кроме гетероскедастичности;

* Немало больших ошибок приходится на диапазон 11-20 повторений, содержащий меньше всего наблюдений.

## Уточнение коэффициентов: зависимость от диапазона

Теперь сделаем поправку для коэффициентов $a,b$ в зависимости от факторных переменных. 

Путём подбора удалось найти две схожие модели (назовём их $b_2$ и $b_3$), немного отличающиеся оценками кросс-валидации на разных диапазонах. **Поскольку на диапазоне повторений 11-20 по-прежнему сохранялись сильные ошибки, было принято решение удалить этот диапазон** (также это можно обосновать тем, что на этом диапазоне меньше данных и сложнее отлавливать выбросы, да и физиологически он не так тесно связан с силой, как другие, о чём уже было сказано).

Далее приведён анализ для модели $RM=MRM \cdot(a+b \cdot Count)$, в которой и $a$ и $b$ имеют поправку на диапазон повторений.

In [ ]:
data %<>%filter(Count<11)
lm(RM~MRM+MRM:Count-1,data) -> b1
b2=lm(RM~MRM:Count:CountGroup+MRM:CountGroup-1,data); b2%>% all()
b3=lm(RM~MRM:Count:CountGroup+MRM:Action-1,data) #%>% all()

Аналогичные выводы:

* На выборочных данных модель ошибается максимум на 18кг и 13%;

* В среднем модель ошибается на 3%;

* В 80% случаев ошибка не превышала 5%;

* Наибольший разброс ошибок приходится на диапазон 4-6;

* Модель статистически значима и удовлетворяет всем нужным требованиям, кроме гетероскедастичности;

## Третье поколение моделей

Путём пошагового отбора переменных были найдены ещё две равнозначные модели ($b_4$ и $b_5$), которые, несмотря на б**о**льшую сложность в сравнении с предыдущими, имеют лучшие оценки при кросс-валидации. 

Статистика для модели $b_5$:

In [ ]:
b4=lm(RM ~ MRM:Action + MRM:I(Count^2) + Action2:I(MRM/Weight) + MRM:Count:CountGroup - 1,data)


b5=lm(RM ~ I((MRM/Index)^6) + MRM:CountGroup + MRM:Action + MRM:CountGroup:Count - 1,data)
b5 %>% all()

Аналогичные выводы:

* На выборочных данных модель ошибается максимум на 13кг и 12%;

* В среднем модель ошибается на менее 3%;

* В 85% случаев ошибка не превышала 5%;

* В целом, на каждом диапазоне имеется почти одинаковый разброс;

* Модель статистически значима и удовлетворяет всем нужным требованиям.

***
## Сравнение линейных моделей и подведение итогов

In [ ]:
kn=c(5,6,7,8,9,10,11,12)

ct=c(8,11)

gr=rep(c('2-10','2-7'),length(kn)) %>% sort(decreasing = T)

m=matrix(nrow=length(kn)*length(ct),ncol=5)

lst=list(b1,b2,b3,b4,b5)


for(i in 1:5){
  
  model=lst[[i]]
  
  for(j in 1:length(ct)){
    dt=data %>% filter(Count<ct[j])
    gl=glm(formula = model$call$formula,data= dt)
    
    getval=function(k){
      boot::cv.glm(dt, gl,K=k)$delta[1] %>% return()
    }
    
    getval.mean=function(k,count){
      map_dbl(1:count,function(x)getval(k)) %>% mean() %>% return()
    }
    
    beg=(j-1)*length(kn)
    
    for(s in 1:length(kn)){
      m[beg+s,i]= getval.mean(kn[s],30)
    }
    #print(m)
  }
  
}

colnames(m)=paste0('b',1:5)
kp=rep(kn,length(ct))

vals=data.frame(kp=rep(kp,5),
                b=as.numeric(m),
                gr=factor(rep(gr,5)),
                n=factor(rep(colnames(m),length(kn)*length(ct)) %>% sort())) %>% 
  tbl_df()


В результате перекрёстной проверки для всех моделей при числе блоков `r kn` было обнаружено, что модель $b_5$ значительно превосходит остальные модели, причём это верно, если делать проверку как на всём диапазоне повторений (2-10), так и на более близком к силовому диапазоне 2-7:

In [ ]:
ggplot(vals,aes(x=kp,y=b,col=n))+theme_bw()+facet_grid(vars(gr), scales="free_y")+
  geom_point(size=4)+geom_line(size=1.)+
  labs(x="Количество блоков при перекрёстной проверке",
       y="Усреднённые значения ошибок после 30 повторных проверок",
       col="Модель",
       title="Оценки качества моделей при перекрёстной проверке",
       subtitle = "Оценка производилась на разных подмножествах данных",
       caption = "Очевидно, что пятая модель превосходит остальные по точности")+
  scale_x_continuous(breaks = kn)+
  theme(legend.position = "bottom")

На втором месте после модели $b_5$ идёт более простая модель $b_3$. 

**Что это за модели?** Посмотрим на модель $b_3$:

In [ ]:
b3 %>% summary()# %>% pander()
cf=b3$coefficients

Она имеет вид $$RM=MRM \cdot (Action_{coef}+CountGroup_{coef} \cdot Count),$$ 
где $Action_{coef}$ равен `r cf[1]` для жима, `r cf[2]` для тяги и `r cf[3]` для приседа, а $CountGroup_{coef}$ есть поправка на диапазон повторений, равная `r cf[4]` для диапазона 2-3, `r cf[5]` -- для 4-6 и `r cf[6]` -- для 7-10. Как видно, первые три коэффициента близки к единице, однако различия между ними имеют значение; заметим также, что поправка на диапазон повторений 2-3 довольно близка к коэффициенту Вендлера (0.0333).

Кстати, по **точности модель $b_3$ превосходит вторую модель Мориса** (ту, что выражена таблицами для каждого движения):

In [ ]:
#модель Мориса

sq = c(1, 1.0475, 1.13, 1.1575, 1.2, 1.242, 1.284, 1.326, 1.368, 1.41)
pr = c(1, 1.035, 1.08, 1.115, 1.15, 1.18, 1.22, 1.255, 1.29, 1.325)
lf = c(1, 1.065, 1.13, 1.147, 1.164, 1.181, 1.198, 1.232, 1.236, 1.24)


rlt = data$MRM * (
  sq[data$Count] * ifelse(data$Action == "Жим", 1, 0) +
    pr[data$Count] * ifelse(data$Action == "Присед", 1, 0) +
    lf[data$Count] * ifelse(data$Action == "Тяга", 1, 0)
)

Show(rlt)

# модель b3
b3 = lm(RM ~ MRM:Count:CountGroup + MRM:Action - 1, data)
b3 %>% predict(data) %>% Show()


Тогда аналогичная таблица, **на сколько нужно умножить свой вес для разного числа повторений**, выглядит так:

In [ ]:
cf = coefficients(b3)

#надо бы исследовать эти числа
count.vec = 2:10 * c(rep(cf[4], 2), rep(cf[5], 3), rep(cf[6], 4))


tibble(
  'Повторы'=1:10,
  'Присед' = c(1, cf[3] + count.vec),
  'Жим' =  c(1, cf[1] + count.vec),
  'Тяга' =  c(1, cf[2] + count.vec)
) %>% kable(caption = 'На сколько нужно умножить свой вес для разного числа повторений, чтобы получить 1ПМ',align = 'cccc')

Модель $b_5$ имеет два дополнительных слагаемых:

In [ ]:
b5 %>% summary()
cf=b5$coefficients

Её вид, аналогично:

$$RM=MRM \cdot (CountGroup_{coef_2} +Action_{coef}+CountGroup_{coef_1} \cdot Count)+coef \cdot\left(\frac{MRM}{Index}\right)^6$$
Здесь $coef$ при последнем слагаемом равен `r sprintf("%.7f", cf[1])`, $Index$ -- индекс массы тела. Дробь внутри последнего слагаемого можно упростить:

$$\frac{MRM}{Index}=\frac{MRM \cdot (0.01 \cdot Height)^2}{Weight}=\frac{MRM \cdot Height^2}{10000\cdot Weight}$$
Вообще *существование этого слагаемого (с шестой степенью) кажется чем-то очень сомнительным, но математика показывает, что это имеет смысл*. **Результаты дисперсионного анализа говорят о том, что
включение указанного коэффициента создаёт значимое отличие между моделями с ним и без него**:

In [ ]:
anova(lm(RM ~  MRM:CountGroup + MRM:Action + 
    MRM:CountGroup:Count - 1, data = data),
      lm( RM ~ I((MRM/Index)^6) + MRM:CountGroup + MRM:Action + 
    MRM:CountGroup:Count - 1, data = data)) %>% pander()

Очень интересно, что в этих двух моделях нет зависимости от возраста, типа телосложения, опыта и пола[^11]. 
Конечно, нельзя рассмотреть всевозможные вариации таких моделей, но среди сотни рассмотренных (в том числе с помощью пакета **caret**) не было обнаружено доказательств значимости включения этих факторов в модель. 

[^11]: по крайней мере, наблюдения от женщин, прошедших опрос, не становились выбросами для этих моделей, а хорошо вписывались во многие наблюдения, принадлежавшие мужчинам

Как итог, **для определения ПМ нужны лишь следующие данные**:

* МПМ;

* Число повторений;

* Движение; 

* Рост и вес спортсмена.


***

## Нелинейные модели

В этом разделе происходит построение **нелинейных моделей** для предсказания повторных максимумов, проверяются формулы [МакГлотина, Ломбарди](https://ru.wikipedia.org/wiki/Одно_повторение_с_максимальным_весом#МакГлотин_(McGlothin)) и другие нелинейные формулы. Оценка качества моделей происходит по аналогичным принципам, то есть значение имеют:

1. точность моделей при перекрёстной проверке, чтобы их можно было сравнить и выбрать лучшую;

1. точность модели на обучающих данных, поскольку неточные модели не имеют пользы, даже если превосходят другие при перекрёстной проверке;

1. простота модели.

Сначала рассматриваются отдельные классы моделей из указанной выше ссылки и мои предложения, затем они сравниваются друг с другом и делаются выводы.

### Основные известные модели

Модели Эпли, Вендлера и О'Коннора -- это одна и та же модель вида $RM=MRM \cdot(1+c \cdot Count)$ с разными значениями коэффициента $c$. Как выяснилось, эти отличия обусловлены только тем, на каком разбросе повторений происходил подбор $c$: значение из формулы О'Коннора наиболее оптимально, если формула должна примерно одинаково хорошо работать на не более чем 20 повторениях, а значение из формулы Эпли, предположительно, оптимально для 2-5 повторений:

In [ ]:
vc=sapply(c(4,8,11,21,50), 
          function(p) nls(RM~MRM*(1+Count*coef),data=data.backup %>% 
                            filter(Count<p),start = list(coef=1/30)) %>% coef())

names(vc)=c(paste(rep("не более чем",4),levels(data$CountGroup)[1:4]) ,"любой")

tibble('Используемый диапазон повторений'=names(vc),'Оценка параметра'=vc) %>% kable(align = 'cc')

Обучив такую модель (с названием $n_1$) для диапазона повторений 2-10, получим следующие результаты:

In [ ]:
ShowSummary=function(model){
  model %>% summary() %>% print()
  
  Show(model %>% predict(data))
}
n1=nls(RM~MRM*(1+Count*coef),data=data.backup %>% filter(Count<4),start = list(coef=1/30))

ShowSummary(n1)

Модели Бжицки и МакГлотина -- это тоже одна и та же модель вида 

$$RM=MRM \cdot \frac{a}{b-Count}$$
с разными $a, b$. Обучив такую модель (с названием $n_2$) для диапазона повторений 2-10, получим более точную модель:

In [ ]:
n2=nls(RM~MRM*a/(b-Count),data,start = list(a=36,b=37))
ShowSummary(n2)

Аналогично модели Мэйхью и Ватана --- это вариации модели

$$RM=\frac{100 \cdot MRM}{a+b\cdot e^{-c\cdot Count}}$$
Обучив такую модель ($n_3$), получим примерно такие же результаты, как у $n_2$:

In [ ]:
n3=nls(RM~100*MRM/(a+b*exp(-c*Count)),data,start = list(a=52,b=42,c=0.055))
ShowSummary(n3)

Наконец, модель Ломбарди имеет вид
$$RM=MRM \cdot Count^a$$
Обучив её ($n_4$), получим примерно те же результаты (с параметром, очень близким к 0.1, предложенным Ломбарди):

In [ ]:
n4=nls(RM~MRM*Count^a,data,start = list(a=0.1))
ShowSummary(n4)

### Новые предложения

Кроме этого, я попробую модель $n_5$:
$$RM=MRM^a \cdot \frac{b}{c-Count+d \cdot Count^2}$$

In [ ]:
n5 = nls(
  RM ~ (MRM^d)*(coef1)/(coef2-Count+c*Count^2),
  data = data,
  start = list(
    coef1 = 50,
    coef2=40,d=1,
    c=0
  )
)
ShowSummary(n5)

и $n_6$:
$$RM=MRM^a \cdot \left(b+c \cdot Count^d \right),$$

In [ ]:
n6 = nls(
  RM ~ MRM ^ vk * (s + coef * Count^kk) ,
  data = data,
  start = list(coef = 0.0333, vk = 1., kk = 1.,s=1)
)
ShowSummary(n6)

а также версии предыдущих шести моделей с поправкой коэффициентов на разные факторные переменные и с включением/отсутствием слагаемого $\left( \frac{MRM}{Index} \right)^6$ (для линейных моделей оно играло роль).

In [ ]:
n7 = nls(
  RM ~ (MRM^d)*(coef1[Action])/(coef2[CountGroup]-Count+c*Count^2),
  data = data,
  start = list(
    coef1 = rep(50,3),
    coef2=rep(40,3),
    d=1,
    c=0
  )
)


n8 = nls(
  RM ~ MRM ^ vk[CountGroup] * (s[Action] + coef * sqrt(Count)) ,
  data = data,
  start = list(coef = 0.0333, vk = rep(1,3),s=rep(1,3))
)


n9=nls(RM~MRM*(
  a[BodyType]/(b[Action]-Count)),data,
  start = list(a=rep(36,3),b=rep(37,3)))

n10=nls(RM~MRM*(
  a[BodyType]/(b[Action]-Count))+d*(MRM/Index)^6,data,
  start = list(a=rep(36,3),b=rep(37,3),d=0))


n11=nls(RM~MRM*(
  a[Action]/(b[CountGroup]-Count)),data,
  start = list(a=rep(36,3),b=rep(37,3)))

n12=nls(RM~MRM*(
  a[Action]/(b[CountGroup]-Count))+d*(MRM/Index)^6,data,
  start = list(a=rep(36,3),b=rep(37,3),d=0))


n13=nls(RM~MRM*(Count^a[CountGroup])+b[Action]*MRM*Count^2,data,start = list(a=rep(0.1,3),b=rep(0.1,3)))

n14=nls(RM~100*MRM/(a[Action]+b*exp(-c[CountGroup]*Count))+d*(MRM/Index)^6,data,start = list(a=rep(52,3),b=42,c=rep(0.0555,3),d=0))


### Сравнение нелинейных моделей

In [ ]:

#cv for nls including errors sample
cv.my2 = function(df = data,
                  fit,
                  start,
                  k = 10,
                  repets = 1) {
  sm = 0
  r = 0
  RM = df$RM
  
  repeat {
    b = T
    while (b) {
      val = tryCatch({
        b = F
        
        blocks = sample.int(k, nrow(df), replace = T)
        
        future.apply::future_sapply(seq(k), function(i) {
          ft = nls(
            formula = fit$call$formula,
            data = df[blocks != i,],
            start = start
          )
          sum((RM[blocks == i] - predict(ft, df[blocks == i,])) ^ 2) %>% return()
        }) %>% sum()
        
      },
      error = function(cond) {
       # print(cond)
        b = T
      })
    }
    
    
    
    sm = sm + val / k
    r = r + 1
    
    if (r == repets) {
      break
    }
    
  }
  
  return(sm / repets)
}

kn = c(5, 6, 7, 8, 9, 10, 11, 12,13,14,15,16,17,18,19,20)

ct = c(8, 11)

gr = rep(c('2-10', '2-7'), length(kn)) %>% sort(decreasing = T)

m = matrix(nrow = length(kn) * length(ct), ncol = 14)

lst = list(n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14)
sts=list(
  list(coef=1/30),
  list(a=36,b=37),
  list(a=52,b=42,c=0.055),
  list(a=0.1),
  list(
    coef1 = 50,
    coef2=40,d=1,
    c=0
  ),
  list(coef = 0.0333, vk = 1., kk = 1.,s=1),
  list(
    coef1 = rep(50,3),
    coef2=rep(40,3),
    d=1,
    c=0
  ),
  list(coef = 0.0333, vk = rep(1,3),s=rep(1,3)),
  list(a=rep(36,3),b=rep(37,3)),
  list(a=rep(36,3),b=rep(37,3),d=0),
  list(a=rep(36,3),b=rep(37,3)),
  list(a=rep(36,3),b=rep(37,3),d=0),
  list(a=rep(0.1,3),b=rep(0.1,3)),
  list(a=rep(52,3),b=42,c=rep(0.0555,3),d=0)
)

for (i in 1:length(lst)) {
  model = lst[[i]]
  
  for (j in 1:length(ct)) {
    dt = data %>% filter(Count < ct[j])
    
    getval.mean = function(k, count) {
      cv.my2(dt, model, sts[[i]], k,count)
    }
    
    beg = (j - 1) * length(kn)
    
    for (s in 1:length(kn)) {
      m[beg + s, i] = getval.mean(kn[s], 30)
    }
  }
  
}

colnames(m) = paste0('n', 1:length(lst))
kp = rep(kn, length(ct))

m[m<100]=NA

vals = data.frame(
  kp = rep(kp, length(lst)),
  b = as.numeric(m),
  gr = factor(rep(gr, length(lst))),
  n = factor(paste0('n',rep(1:14, length(kn) * length(ct)) %>% sort()))
) %>%
  tbl_df()

Сравним названные модели, сделав перекрёстную проверку на диапазоне повторений 2-10:

In [ ]:
(ggplot(vals %>% filter(gr=="2-10"), aes(x = kp, y = b, col = n)) + theme_bw() +
  geom_point(size = 4) + geom_line(size = 1.) +
  labs(
    x = "Количество блоков при перекрёстной проверке",
    y = "Усреднённые значения ошибок после 30 повторных проверок",
    col = "Модель",
    title = "Оценки качества моделей при перекрёстной проверке"
  ) +
  scale_x_continuous(breaks = kn) +
  theme(legend.position = "right")) %>% ggplotly()


Как видно, самой лучшей моделью оказалась $n_{14}$ (основанная на моделях Мэйхью и Ватана), на втором месте идёт $n_8$ (моё предложение):

In [ ]:
n14 %>% ShowSummary()

n8 %>% ShowSummary()

Обе модели имеют одинаковую среднюю ошибку в менее чем 2.9%. По точности они схожи с линейными моделями $b_3,b_5$.


In [ ]:
cv.my3 = function(df = data,
                  fit,
                  start=NULL,
                  k = 10,
                  repets = 1) {
  sm = 0
  r = 0
  RM = df$RM
  
  repeat {
    b = T
    while (b) {
      val = tryCatch({
        b = F
        
        blocks = sample.int(k, nrow(df), replace = T)
        
        if(is.null(start)){
          
          future.apply::future_sapply(seq(k), function(i) {
            ft = lm(
              formula = fit$call$formula,
              data = df[blocks != i,]
            )
            sum((RM[blocks == i] - predict(ft, df[blocks == i,])) ^ 2) %>% return()
          }) %>% sum()
          
        }else{
          
          future.apply::future_sapply(seq(k), function(i) {
          ft = nls(
            formula = fit$call$formula,
            data = df[blocks != i,],
            start = start
          )
          sum((RM[blocks == i] - predict(ft, df[blocks == i,])) ^ 2) %>% return()
        }) %>% sum()        
        }

        
      },
      error = function(cond) {
        b = T
      })
    }
    
    
    
    sm = sm + val / k
    r = r + 1
    
    if (r == repets) {
      break
    }
    
  }
  
  return(sm / repets)
}

kn = c(5, 6, 7, 8, 9, 10, 11, 12,13,14,15,16,17,18,19,20)

ct = c(8, 11)

gr = rep(c('2-10', '2-7'), length(kn)) %>% sort(decreasing = T)

m = matrix(nrow = length(kn) * length(ct), ncol = 4)

lst = list(n8,n14,b3,b5)
sts=list(
  list(coef = 0.0333, vk = rep(1,3),s=rep(1,3)),
  list(a=rep(52,3),b=42,c=rep(0.0555,3),d=0),
  NULL,
  NULL
)

for (i in 1:length(lst)) {
  model = lst[[i]]
  
  for (j in 1:length(ct)) {
    dt = data %>% filter(Count < ct[j])
    
    getval.mean = function(k, count) {
      cv.my3(dt, model, sts[[i]], k,count)
    }
    
    beg = (j - 1) * length(kn)
    
    for (s in 1:length(kn)) {
      m[beg + s, i] = getval.mean(kn[s], 40)
    }
  }
  
}

colnames(m) = c("n8","n14","b3","b5")
kp = rep(kn, length(ct))

#m[m<100]=NA

vals = data.frame(
  kp = rep(kp, length(lst)),
  b = as.numeric(m),
  gr = factor(rep(gr, length(lst))),
  n = factor(
    c("n8" %>% rep(length(ct)*length(kn)),
      "n14"%>% rep(length(ct)*length(kn)),
      "b3"%>% rep(length(ct)*length(kn)),
      "b5"%>% rep(length(ct)*length(kn)))
    )
) %>%
  tbl_df()

Если сравнить лучшие найденные линейные и нелинейные модели, окажется, что модели $n_8$ и $b_3$ в целом неотличимы, а модель $n_{14}$ немного превосходит $b_5$:

In [ ]:
(ggplot(vals %>% filter(gr=="2-10"), aes(x = kp, y = b, col = n)) + theme_bw() +
    geom_point(size = 4) + geom_line(size = 1.) +
    labs(
      x = "Количество блоков при перекрёстной проверке",
      y = "Усреднённые значения ошибок после 40 повторных проверок",
      col = "Модель",
      title = "Оценки качества моделей при перекрёстной проверке"
    ) +
    scale_x_continuous(breaks = kn) +
    theme(legend.position = "right")) %>% ggplotly()

# Выводы

В результате проведённого исследования была найдена модель, не зависящая от типа телосложения, опыта тренировок и многих других характеристик, но при этом дающая хорошие результаты на 85% данных. 

Я уверен, что большие погрешности в единичных случаях -- это та часть закономерности, зависящая от человека и многих неучтённых характеристик. Тем не менее, влияние этих неучтённых факторов далеко не так велико, как предполагали многие: намного больше повторный максимум зависит от обычной математики.

Было продемонстрировано, что для числа повторений выше 10 нельзя построить достаточно точную модель (по крайней мере, при текущем наборе данных), но для диапазона 2-10 -- ещё как можно.

Диапазон выше 10 повторений был отсеян по следующим причинам:

1. физиологически менее сильная связь между 1ПМ и большим числом повторений;

1. меньшая детерминированность между 1ПМ и большим числом повторений, обнаруженная при разведочном анализе данных, отчего трудно выявить достоверность данных и построить достаточно точную модель;

1. маленький объём данных, разбросанных очень неравномерно на большом диапазоне (от 11 до 35).

Посчитать собственные результаты при помощи найденной модели можно [здесь](https://dmitrypasko.shinyapps.io/RMbyMRMestimating/).

***
# Возможности для применения

## Оценка наличия прогресса в тренировках

Теперь практически в любой момент тренировочного процесса[^1] **спортсмен может оценить свой ПМ, не делая проходку**. Отслеживание ПМ помогает оценить близость к намеченной цели и вообще наличие прогресса: иногда обстоятельства мешают уложиться в цикл (особенно если это третий цикл подряд) и вместо семи повторений спортсмен может сделать только 5 --- считаются ли эти пять повторений прогрессом? Так или иначе, можно сделать перерыв и начать новый цикл, основываясь уже на новых данных[^2].

[^1]: разумеется, после тех тренировок, где происходит максимальное усилие: часто силовые циклы делят тренировки на тяжёлые и лёгкие, силовые и скоростные и т. п., но максимальное усилие, позволяющее оценить ПМ, совершается именно на тяжёлых силовых тренировках (и то -- не всегда)
[^2]: но также можно несколько тренировок уделить подсобным или родственным упражнениям, а потом вернуться к прежнему циклу

## Выражение многоповторного максимума через многоповторный

Здесь всё интуитивно понятно. Для человека, например, моей комплекции (рост 170, вес 70) для приседа можно составить такую таблицу (исходя из модели $b_5$):

In [ ]:
f=function(MRM,Count,Action='Присед',Weight=70,Height=170){
  
  if(Count<2){
    Count=2
  } else if(Count>10){
    Count=10
  }
  
  act=factor(Action,levels = levels(data$Action))
  
  up=c(4,8,11)
  lv=levels(data$CountGroup)
  
  cg=lv[Count<up] %>% first() %>% factor(levels=lv)
  
  df=data.frame(MRM,
                Count,
                Action=act,
                CountGroup=cg,
                Index=Weight/(0.01*Height)^2)
  
  predict(b5,df,se.fit = T,interval = "confidence",level=0.95)[[1]] %>% return()
  
}


mrm=function(RM, count){
  
  s=function(MRM) abs(RM-f(MRM=MRM,Count=count, Action = 'Присед')[1]) %>% return()
  
  optim(par=c(0.8*RM),
        fn=s,
        lower=0.6*RM,
        upper = RM,
        method="Brent")$par %>% return()
}

cf=coefficients(b5)
mrm3=function(RM,count,Action='Присед',Weight=70,Height=170){
  
  ctg=3
  if(count<7){ctg=2}
  if(count<4){ctg=1}
  
  act=0
  if(Action=="Тяга"){
    act=cf[5]}else if(Action=="Присед"){act=cf[6]}
  
  
  polyroot(c(-RM,cf[1+ctg]+count*cf[6+ctg]+act,0,0,0,0,cf[1]*((0.01*Height)^2/Weight)^6))[1] %>% Re()
  
}
#mrm(150,3)


vec=seq(100,300,length.out = 81)

m=matrix(nrow=length(vec),ncol=10)
m[,1]=vec
for(i in 2:10){
  for(j in 1:length(vec))
    m[j,i]=mrm3(m[j,1],i)
}

colnames(m)=c('1 rep',paste(2:10, 'reps') %>% as.character())

m %>% round(2) %>% tbl_df()

В этой таблице **каждая строка обозначает одно и то же усилие, выраженное разным числом повторений**. Если для человека верна какая-то ячейка (вес на число повторений), должна быть верна и вся строка.

Также тут можно проследить такую возможность: если в режиме на, например, 5 повторений вы взяли 80кг (80х5 эквивалентно `r f(MRM=80,Count=5)[1]`), но смогли сделать с этим весом 8 повторений (что эквивалетно `r f(MRM=80,Count=8)[1]`), то какой вес нужно взять, чтобы делать прежние 5 повторений? Из формулы следует, что `r mrm3(f(MRM=80,Count=8)[1],5)`.

## Постановка эквивалентной цели

Если вы ходите поднять свой $RM$ на 15 кг, не обязательно ставить цель "в конце цикла поднять $RM+15$ кг на раз". Зная $RM+15$, можно рассчитать свой целевой 2ПМ, 3ПМ или 4ПМ и стремиться к концу цикла достичь именно тех величин. 

***
## Возможности для применения в анализе силовых циклов

Известно, что невозможно постоянно увеличивать вес отягощений, так как рано или поздно спортсмен упрётся в *тренировочное плато* и может дойти до перетренированности, истощиться, заболеть, а то и получить травму, в результате чего произойдёт снижение силовых показателей. Основной способ обойти эту проблему и многие годы увеличивать силовые показатели --- это **циклирование** (периодизация нагрузок). При этом **даже сейчас циклы строятся больше на основе опыта (методом проб и ошибок) и интуиции, чем с использованием каких-то научных знаний**. И хоть за последние полвека уже тысячи спортсменов доказали пользу нескольких моделей циклирования и разнообразия нагрузок, текущая ситуация далека от идеальной. Я считаю, что построенная ранее модель (или аналогичная ей) может сделать в этом направлении шаг вперёд.

Если раньше многие циклы записывались от силы таблицами в Excel и не всегда были понятны, то *использование модели, отождествляющей повторный максимум с многоповторным, позволяет визуализировать зависимость "максимального усилия" от дня тренировки*, то есть **отождествить цикл с временным рядом**. Дальнейшие исследования таких временных рядов позволит разработать циклы, близкие к идеальным, или хотя бы обнаружить ту составляющую в циклах, за счёт которых они работают, и отсеять те программы тренировок, которые не будут работать.

Таким образом, в будущем ***тренировочный процесс можно будет представить как последовательность нагрузок в разных режимах (в основном для тренировки разных мышечных волокон), которые требуют усилий, подчиняющихся определённым математически выраженным закономерностям***.

Далее приводятся примеры визуализации нескольких циклов.

### Простейший цикл

In [ ]:
library(tidyverse)
library(magrittr)
library(ggformula)
library(ggalt)


mx=function(vals,count, withcomp=T){
  
  t=1:length(vals) %>% map_dbl(function(i) f(MRM=vals[i],Count=count[i],Action='Жим')[1])
  
  #t=vals*(1+0.0333*count)
  t[count==1]=vals[count==1]
  if(withcomp){
      s=t[length(t)]*(1+runif(1,0.,0.005))
  return(c(t,s))
  }
  return(t)
}


x=c(100,120,115,105,130,120,125)
y=c(10,4,6,10,3,6,5)

vc=c(mx(x,y),mx(x+4,y),mx(x+9,y))
lb=paste0(c(x,vc[8],x+4,vc[16],x+9,vc[24]) %>% round(),"x",c(y,1,y,1,y,1))

Возьмём очень простой цикл из тех, которые можно иногда применять в межсезонье. Он состоит из `r length(x)` тренировок, на которых самый тяжёлый подход составляет (при конкретных цифрах) `r paste0(x,'x',y)` соответственно.

Если прогнать такой цикл через модель (в случае с приседом), получим такую динамику:

In [ ]:
tb=tibble(d=1:length(vc),val=vc) %>% mutate(day=factor(ifelse(d%%(length(x)+1)==0,"проходка","тренировка")))

gg=ggplot(tb,aes(x=d,y=val))+
  geom_hline(yintercept = 141,size=1.,linetype="dashed",alpha=0.8)+
  geom_hline(yintercept = vc[length(vc)],size=1.,linetype="dashed",alpha=0.8)+
  geom_xspline(size=1,spline_shape = -0.3,linetype="dotdash",col="green",alpha=0.9)+
  geom_point(aes(col=day),size=4)+
  annotate("text", x = 22, y = 139.7, label = "Начальные результаты") +
  annotate("text", x = 17, y = 153.2, label = "Итоговые результаты") +
  annotate("text", x = tb$d+0.4, y = tb$val-ifelse(seq(tb$d)%%8!=0,0.8,0.3), label = lb) +
  labs(title='Динамика простого цикла (три прохода)',
       subtitle="Использование модели помогает отследить прогрессию нагрузок\nЗелёную кривую можно рассматривать как аппроксимацию уровня тренированности",
       x="Номер тренировки",
       y="Требуемое усилие",
       caption="Не используйте такой цикл больше 2-3 проходов подряд, он может привести к перетренированности") +
  guides(color=guide_legend(title="Тип дня"))+
  theme_bw()+theme(legend.position = c(0.85,0.15))

print(gg)

(gg) %>% ggplotly()#+ guides(color=FALSE)
  


### Пиковый цикл Верхошанского

[Ссылка на цикл](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/blob/master/Материалы%20по%20пауэрлифтингу%20и%20не%20только/циклы/Пиковый%20цикл%20Верхошанского.xls)

In [ ]:
#155 верхошанский
x=c(100,125,107.5,132.5,120,140,127.5,147.5,117.5,132.5,125,162.5)
y=c(6,5,5,4,3,3,3,2,5,2,2,1)
vc=c(mx(x,y,F),mx(x+8,y,F),mx(x+15,y,F))
lb=paste0(c(x,x+8,x+15) %>% round(),"x",c(y,y,y))

tb=tibble(d=1:length(vc),val=vc) %>% mutate(day=factor(ifelse(d%%length(x)==0,"проходка","тренировка")))

gg=ggplot(tb,aes(x=d,y=val))+
  geom_hline(yintercept = 155,size=1.,linetype="dashed",alpha=0.7)+
  geom_hline(yintercept = vc[length(vc)],size=1.,linetype="dashed",alpha=0.7)+
  geom_xspline(size=1,spline_shape = -0.7,linetype="dotdash",col="green",alpha=0.9)+
  geom_point(aes(col=day),size=4)+
  annotate("text", x = 5.5, y = 159, label = "Начальные результаты") +
  annotate("text", x = 4.5, y = 176, label = "Итоговые результаты") +
  annotate("text", x = tb$d+0.4, y = tb$val-ifelse(seq(tb$d)%%length(x)!=0,0.8,0.3), label = lb) +
  labs(title='Пиковый цикл Верхошанского (три прохода)',
       subtitle="Медленный подъём чуть выше бывшего максимума, затем отдых (разгон) и проходка",
       x="Номер тренировки",
       y="Требуемое усилие",
       caption="Файл с циклом прилагается") +
  guides(color=guide_legend(title="Тип дня"))+
  theme_bw()+theme(legend.position = c(0.85,0.2))

print(gg)

(gg)%>% ggplotly()#+ guides(color=FALSE) 




### Цикл тома МакКаллоу

[Ссылка на цикл](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/blob/master/Материалы%20по%20пауэрлифтингу%20и%20не%20только/циклы/Tyagovy-tsikl-Toma-Makkelou.xls)

In [ ]:
x=c(110,112,117,122,126,131,136,141,145,150,155,160,166,177,120,122.5,128,133,138,143,149,154,159,165,170,175,182,195,134,136,142,148,154,160,165,171,177,183,190,195,203,216)
y=rep(c(10,10,8,8,5,5,5,5,3,3,2,2,1,1),3)
vc=c(mx(x,y,F))
ct=length(y)/3

#plot(vc,type = "b")

lb=paste0(x ,"x", y)

tb=tibble(d=1:length(vc),val=vc) %>% mutate(day=factor(ifelse(d%%ct==0,"проходка","тренировка")))

gg=ggplot(tb,aes(x=d,y=val))+
  geom_hline(yintercept = 160,size=1.,linetype="dashed",alpha=0.7)+
  geom_hline(yintercept = vc[length(vc)],size=1.,linetype="dashed",alpha=0.7)+
  geom_xspline(size=1,spline_shape = -0.7,linetype="dotdash",col="green",alpha=0.9)+
  geom_point(aes(col=day),size=4)+
  annotate("text", x = 35, y = 162, label = "Начальные результаты") +
  annotate("text", x = 35, y = 214, label = "Итоговые результаты") +
  annotate("text", x = tb$d+0.4, y = tb$val-ifelse(seq(tb$d)%%ct!=0,0.8,0.3), label = lb) +
  labs(title='Тягловый цикл Тома Маккелоу (три прохода)',
       subtitle="Колебание, подъём, колебание, проходка",
       x="Номер тренировки",
       y="Требуемое усилие",
       caption="Файл с циклом прилагается. Первый день цикла сдвинут вниз для удобства визуализации") +
  guides(color=guide_legend(title="Тип дня"))+
  theme_bw()+theme(legend.position = c(0.85,0.15))

print(gg)

(gg) %>% ggplotly()#+ guides(color=FALSE)


# Полезные ссылки

[Сайт, где можно опробовать модель на себе](https://dmitrypasko.shinyapps.io/RMbyMRMestimating/)

[Русскоязычный опрос](https://forms.gle/e428M7osMCA98ANV9), [англоязычный опрос](https://forms.gle/kbwqQtq8vWpKdJbH8). Если данных станет значительно больше, исследование повторится.

[Мой гитхаб](https://github.com/PasaOpasen)

[Раздел этой статьи на GitHub, исходные датасеты, коды](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/tree/master/Estimating%20RM)

[Книги и материалы по пауэрлифтингу](https://github.com/PasaOpasen/Powerlifting-training-diary-and-articles/tree/master/Материалы%20по%20пауэрлифтингу%20и%20не%20только)

**Моя электронная почта**: qtckpuhdsa@gmail.com

# Литература

[1] Хэтфилд Ф. К., Всестороннее руководство по развитию силы // Красноярск: Союзспорт, 1992. - 284 с.

[2] Вадим Протасенко. Думай! Или «Супертренинг» без заблуждений.

[3] Уикем Х., Гроулмунд Г. Язык R в задачах науки о данных: импорт, обработка, визуализация и моделирование данных. : Пер. с англ. --- СПб. : ООО "Диалектика", 2018. --- 592 с.

[4] Мастицкий С. Э., Шитиков В. К. Статистический анализ и визуализация данных с помощью R. -- М.: ДМК Пресс, 2015. -- 496 с.

[5] Роберт И. Кабаков. R в действии. Анализ и визуализация данных в программе R. / пер. с англ. Полины А. Волковой. -- М: ДМК Пресс, 2016. -- 588 с.

[6] Джеймс Г., Уиттон Дм Хасти Т., Тибширани Р. Введение в статистическое обучение с примерами на языке R. Пер. с англ. С. Э. Мастицкого - М.: ДМК Пресс, 2016. - 450 с: 

[7] Крупкина, Т. В. Математическая статистика [Электронный ресурс] : курс лекций / Т. В. Крупкина, А. К. Гречкосеев. – Электрон. дан. (3 Мб). – Красноярск : ИПК СФУ, 2009.

[8] Brandon Lilly «The Cube method» (2012)
